**Import the necessary modules**

In [ ]:
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

**Define the surface as a neural net**

In [ ]:
class LSTMForgetBlock(tf.keras.layers.Layer):
    def __init__(self, num_nodes, dtype=tf.float32):
        super().__init__(name='LSTMForgetBlock', dtype=dtype)
        self.W_f = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='W_f', use_bias=False)
        self.U_f = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='U_f')
        self.W_g = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='W_g', use_bias=False)
        self.U_g = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='U_g')
        self.W_r = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='W_r', use_bias=False)
        self.U_r = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='U_r')
        self.W_s = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='W_s', use_bias=False)
        self.U_s = tf.keras.layers.Dense(num_nodes, dtype=dtype, name='U_s')

    def call(self, x, h, c):
        f = tf.keras.activations.tanh(self.W_f(x) + self.U_f(h))
        g = tf.keras.activations.tanh(self.W_g(x) + self.U_g(h))
        r = tf.keras.activations.tanh(self.W_r(x) + self.U_r(h))
        s = tf.keras.activations.tanh(self.W_s(x) + self.U_s(h))
        c = f*c + g*s
        return r*tf.keras.activations.tanh(c), c


class LSTMForgetNet(tf.keras.models.Model):
    """
    Description: 
        LSTM Forget architecture
    Args:
        num_nodes: number of nodes in each LSTM layer
        num_layers: number of LSTM layers
    """
    def __init__(self, num_nodes, num_layers, dtype=tf.float32, name = 'LSTMForgetNet'):
        super().__init__(dtype=dtype, name=name)
        self.num_nodes = num_nodes
        self.num_layers = num_layers
        self.lstm_layers = [LSTMForgetBlock(num_nodes, dtype=dtype) for _ in range(num_layers)]
        self.final_dense = tf.keras.layers.Dense(units=1, activation=None, dtype=dtype)
        self.batch_norm = tf.keras.layers.BatchNormalization(axis=1)
        
        

    def call(self, *args):
        x = tf.concat(args, axis=1)
        h = tf.zeros_like(x)
        c = tf.zeros((x.shape[0], self.num_nodes), dtype=self.dtype)
        for i in range(self.num_layers):
            h, c = self.lstm_layers[i](x, h, c)
            h = self.batch_norm(h)
            c = self.batch_norm(c)
        y = self.final_dense(h)
        return y


surface = LSTMForgetNet(num_nodes=50, num_layers=3)
mu = LSTMForgetNet(num_nodes=20, num_layers=3)

**Define the area functional**

In [ ]:
@tf.function
def minimal_op(u, r, t):
  with tf.GradientTape(persistent=True) as tape:
    tape.watch([r, t])
    u_ = u(r, t)
    u_r, u_t = tape.gradient(u_, [r, t])
  return tf.sqrt(r**2 * (1.0 + u_r**2) + u_t**2)
    

**Define the boundary condition**

In [ ]:
t_boundary = [0., 4.0 * np.pi]

def helix_boundary(u, t):
  return u(tf.ones_like(t), t) - t

**Define domain and boundary samplers**

In [ ]:
def domain_sampler(n_sample, low=[0., t_boundary[0]], high=[1., t_boundary[1]]):
  r = tf.random.uniform(shape=(n_sample, 1), minval=low[0], maxval=high[0])
  t = tf.random.uniform(shape=(n_sample, 1), minval=low[1], maxval=high[1])
  return r, t 

def boundary_sampler(n_sample, low=t_boundary[0], high=t_boundary[1]):
  t = tf.random.uniform(shape=(n_sample, 1), minval=low, maxval=high)
  return t

**Define the loss function and the training procedure**

In [ ]:
def loss_surface(network, mul, r, t, t_b, rho):
  return tf.reduce_mean(minimal_op(network, r, t)) + tf.reduce_mean(helix_boundary(network, t_b) * mul(tf.ones_like(t_b), t_b)) \
         + 0.5 * rho * tf.reduce_mean(helix_boundary(surface, t_b)**2)

def loss_mu(network, mul, mu_0, t_b, rho):
  return tf.reduce_mean((mul(tf.ones_like(t_b), t_b) - mu_0 - rho * helix_boundary(network, t_b))**2)
  

@tf.function
def train_step_surface(optimizer, network, mul, r, t, t_b, rho):
  with tf.GradientTape() as tape:
    L = loss_surface(network, mul, r, t, t_b, rho)
  grads = tape.gradient(L, network.trainable_weights)
  optimizer.apply_gradients(zip(grads, network.trainable_weights))
  return L

@tf.function
def train_step_mu(optimizer, network, mul, t_b, rho):
  mu_0 = mu(tf.ones_like(t_b), t_b)
  with tf.GradientTape() as tape:
    L = loss_mu(network, mul, mu_0, t_b, rho)
  grads = tape.gradient(L, mul.trainable_weights)
  optimizer.apply_gradients(zip(grads, mul.trainable_weights))
  return L

def learn(network, mul, rho, aleph, epochs=10000, n_sample=1000):
  r, t = domain_sampler(n_sample)
  t_b = boundary_sampler(n_sample)
  learning_rate_1 = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000, 2000, 10000], [5e-3, 1e-3, 5e-4, 1e-4])
  optimizer_1 = tf.keras.optimizers.Adam(learning_rate_1)
  learning_rate_2 = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000, 2000, 10000], [5e-3, 1e-3, 5e-4, 1e-4])
  optimizer_2 = tf.keras.optimizers.Adam(learning_rate_2)
  print("{:>6}{:>12}{:>12}{:>18}".format('Epoch', 'Loss_1', 'Loss_2', 'Runtime(s)'))
  start = time.time()
  for epoch in range(epochs):
    L_1 = train_step_surface(optimizer_1, network, mul, r, t, t_b, rho)
    for i in range(1):
      L_2 = train_step_mu(optimizer_2, network, mul, t_b, rho)
    rho *= aleph
    if epoch % 10 == 0:
      print('{:6d}{:12.6f}{:16.6f}{:12.4f}'.format(epoch, L_1, L_2, time.time()-start))
      r, t = domain_sampler(n_sample)
      t_b = boundary_sampler(n_sample)
      surface.save_weights('surface_{}'.format(epochs))

**Learn the solution**

In [ ]:
surface(*domain_sampler(10)) # build the network
mu(*domain_sampler(10))
epochs = 500
n_sample = 1000
rho = 50.
aleph = 1.004
learn(surface, mu, rho, aleph, epochs, n_sample)
surface.save_weights('surface_{}'.format(epochs))

 Epoch      Loss_1      Loss_2        Runtime(s)


**Visualize the solution**

In [ ]:
def plot_solutions(learned, resolution=30):
  fig = plt.figure(figsize=(16, 8))
  ax_l = fig.add_subplot(121, projection='3d')
  ax_t = fig.add_subplot(122, projection='3d')
  r = np.linspace(0., 1., num=resolution, endpoint=True)
  t = np.linspace(t_boundary[0], t_boundary[1], num=resolution, endpoint=True)
  r, t = np.meshgrid(r, t)
  x = r * np.cos(t)
  y = r * np.sin(t)
  z_l = learned(r.reshape(-1, 1), t.reshape(-1, 1)).numpy()
  z_t = t.reshape(-1, 1)
  grid = (resolution, resolution)
  #x = x.reshape(grid)
  #y = y.reshape(grid)
  z_l = z_l.reshape(grid)
  z_t = z_t.reshape(grid)
  ax_l.plot_surface(x, y, z_l, color='deeppink')
  ax_l.set_title('learned minimal surface', fontsize=15)
  ax_t.plot_surface(x, y, z_t, color='blue')
  ax_t.set_title('helicoid', fontsize=15)
  plt.tight_layout()
  plt.show()

plot_solutions(learned=lambda r, t: surface(r, t))

In [ ]:
surface.summary()